In [1]:
import csv
import time
import multiprocessing as mp
import argparse
import warnings
import numpy as np
import librosa
from tqdm import tqdm

import torch
from torch import nn
from torch.utils import data
from torch.optim.lr_scheduler import ReduceLROnPlateau, CosineAnnealingLR
from tensorboardX import SummaryWriter
from torchvision.utils import make_grid

from utils.radam import RAdam,Lookahead
from utils.aug_utils import custom_stft_aug
from utils.stft_utils.stft import STFT
from utils.utils import *
from utils.loss_utils import CosineDistanceLoss

from dataloader import *
from efficientunet import *
from efficientunet.layers import BatchNorm2d,BatchNorm2dSync

## 여기서 원래의 wav 파일들을 로드하는 방식으로 변경!!!!! 

class Dataset(torch.utils.data.Dataset):
    def __init__(self,X):
        self.data = X
    def __len__(self):
        return len(self.data)
    def __getitem__(self,idx):
        if isinstance(self.data[idx],str):
            X = np.load(self.data[idx])
        else:
            X = self.data[idx]
        
        X = stft_process(X) ## 2,F,T mag,phase
        return X
    
model = get_efficientunet_b2(out_channels=3, concat_input=True, pretrained=False, bn = BatchNorm2d)
model.load_state_dict(torch.load('./models/masked/exp6-re/best_5761.pth'))
# model = nn.DataParallel(model)
model.cuda()
model.eval()

EfficientUnet(
  (encoder): Encoder(
    (stem_conv): Conv2dSamePadding(2, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
    (stem_batch_norm): BatchNorm2d(32, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
    (stem_swish): Swish()
    (blocks): ModuleList(
      (0): MBConvBlock(
        (swish): Swish()
        (_depthwise_conv): Conv2dSamePadding(32, 32, kernel_size=(3, 3), stride=[1, 1], groups=32, bias=False)
        (_bn1): BatchNorm2d(32, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
        (_se_reduce): Conv2dSamePadding(32, 8, kernel_size=(1, 1), stride=(1, 1))
        (_se_expand): Conv2dSamePadding(8, 32, kernel_size=(1, 1), stride=(1, 1))
        (_project_conv): Conv2dSamePadding(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (_bn2): BatchNorm2d(16, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
      )
      (1): MBConvBlock(
        (swish): Swish

In [2]:
%%time
paths = load_stft_unlabel_datas_path()

CPU times: user 896 ms, sys: 596 ms, total: 1.49 s
Wall time: 1.53 s


In [3]:
import os
save_basepath = '../eggdata/TrainSTFT/pseudo_label/6-re-5761/'
os.makedirs(save_basepath,exist_ok=True)
dsets = ['DSD100', 'KSS', 'zeroth_korean', 'speech_ko']
for d in dsets:
    os.makedirs(save_basepath+d,exist_ok=True)

_paths = []
save_paths = [] 
for path in tqdm(paths):
    spaths = save_basepath + '/'.join(path.split('/')[-2:])
    if not os.path.exists(spaths):
        save_paths.append(spaths)
        _paths.append(path)
paths = _paths

100%|██████████| 725202/725202 [06:16<00:00, 1927.84it/s]


In [4]:
print(len(paths))
print(len(save_paths))

46533
46533


In [5]:
dataset = Dataset(paths)
batch_size = 250
loader = data.DataLoader(dataset=dataset,
                                 batch_size=batch_size,
                                 num_workers = mp.cpu_count(),
                                 shuffle=False)

In [ ]:
Ys = []
cnt = 0
for i,X in enumerate(tqdm(loader)):
    X = X.cuda()
    with torch.no_grad():
#         print(X.shape)
        Y = model(X) ## B,3,F,T
        X = X.cpu().detach().numpy()
        Y = Y.cpu().detach().numpy()
        for j in range(len(Y)):
            np.save(save_paths[i*batch_size + j],Y[j])
#     Ys.append(Y) ## N,3,F,T

 92%|█████████▏| 172/187 [07:16<00:33,  2.26s/it]